In [ ]:
%run map_coloring.py

In [ ]:
show_map({county: '-' for county in counties}, show_png=False)

## Random Coloring

In [ ]:
num_colors = 5
colors = {county: rand.randint(1, num_colors) for county in counties}
show_map(colors)

## Greedy Coloring

In [ ]:
def run_step(state, show=False):
    running = True
    index = state['index']
    if index < len(state['counties']):
        county = state['counties'][index]
        used_colors = [state['colors'][nbr] for nbr in nbrs[county]]
        c = 1
        while c in used_colors:
            c += 1

        print(f'{index + 1:2d}: {county_names[county]:12s} -> color {c}')
        state['colors'][county] = c

        state['index'] += 1

    if show:
        show_map(state['colors'], show_png=False)
    return state['index'] < len(state['counties'])

In [ ]:
state = dict(colors=defaultdict(int), index=0, counties=sorted(counties))

In [ ]:
# Repeatedly run this cell.
run_step(state, show=True)

In [ ]:
state = dict(colors=defaultdict(int), index=0, counties=sorted(counties))
while True:
    if not run_step(state, show=False):
        break
show_map(state['colors'])